<a href="https://colab.research.google.com/github/itekkie/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:15 http://arc

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Vine_Review_Analysis").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_FR_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         FR|      14952|R32VYUWDIB5LKE|0552774294|     362925721|    The God Delusion|               Books|          5|            0|          0|   N|                Y|a propos de ce livre|je conseille fort...| 2013-02-13|
|         FR|      14952|R3CCMP4EV6HAVL|B004GJXQ20|     268067011|A Game of Thrones...|Digital_Ebook_Pur

### Create vine_df

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R32VYUWDIB5LKE|          5|            0|          0|   N|                Y|
|R3CCMP4EV6HAVL|          5|            0|          0|   N|                Y|
|R14NAE6UGTVTA2|          3|            1|          3|   N|                Y|
|R2E7QEWSC6EWFA|          4|            0|          1|   N|                Y|
|R26E6I47GQRYKR|          2|            3|          5|   N|                N|
|R1RJMTSNCKB9LP|          2|            0|          0|   N|                Y|
|R2P2XF84YELQBZ|          5|            0|          3|   N|                Y|
| RJKULSX2Y5R07|          5|            0|          0|   N|                Y|
|R3UYE0U7SQCI8Q|          5|            2|          3|   N|                Y|
|R1TKJ7XFS3RDEB|          5|            0|          1|   N|     

In [5]:
# Step 1. Filter the data and create a new DataFrame where the total_votes count is equal to or greater than 20
ttl_g20_vine_df = vine_df.filter(vine_df["total_votes"]>20)
ttl_g20_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2LX6LJ7BA92OO|          1|           21|         27|   N|                Y|
|R3VMEW1JWIXV1E|          1|            2|         26|   N|                N|
| R8RBQTRZ32CFG|          1|            0|         29|   N|                Y|
|R2612UZ2GU2K29|          5|           25|         32|   N|                Y|
|R28X8YI8B00C6S|          4|           52|         63|   N|                Y|
|R3O3RDGDHHINF2|          5|           41|         53|   N|                Y|
|  RQRAR269EQNP|          2|            7|         35|   N|                Y|
|R1FSDCDV0XV3X6|          4|           27|         33|   N|                Y|
|R2LK33YEMGHB27|          5|            4|         28|   N|                Y|
| RRZXAHEVTT2HC|          5|           69|         92|   N|     

In [6]:
# Step 2. Filter the ttl_g20_vine_df and create a new DF where helpful_votes / total_votes >= 50%.
helpful_ttl_g20_vine_df = ttl_g20_vine_df.filter((ttl_g20_vine_df["helpful_votes"]/ttl_g20_vine_df["total_votes"]) >=0.5)
helpful_ttl_g20_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2LX6LJ7BA92OO|          1|           21|         27|   N|                Y|
|R2612UZ2GU2K29|          5|           25|         32|   N|                Y|
|R28X8YI8B00C6S|          4|           52|         63|   N|                Y|
|R3O3RDGDHHINF2|          5|           41|         53|   N|                Y|
|R1FSDCDV0XV3X6|          4|           27|         33|   N|                Y|
| RRZXAHEVTT2HC|          5|           69|         92|   N|                Y|
|R3AUV7LA2MZTEO|          5|           16|         21|   N|                N|
| R3NOR8QP5W657|          1|           15|         29|   N|                N|
|R1XM8IQUDFKBQZ|          2|           19|         27|   N|                N|
| RZR9JFL71VIHX|          5|           53|         62|   N|     

In [7]:
# Step 3. Filter the helpful_ttl_g20_vine_df and create a new DF where a review is the Vine program (paid), vine == 'Y'
paid_helpful_ttl_g20_vine_df = helpful_ttl_g20_vine_df.filter(helpful_ttl_g20_vine_df["vine"] == "Y")
paid_helpful_ttl_g20_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R22SUWPP3KRZT6|          3|           21|         26|   Y|                N|
|R30LQPCSI3ELHN|          5|           18|         23|   Y|                N|
|R3QIS7D8004AD2|          3|           76|         80|   Y|                N|
|R3R6CDRZU5U2XW|          4|           41|         47|   Y|                N|
|R16J9CTJQ41IC2|          5|           18|         22|   Y|                N|
| RJNGHTZD3NT6F|          3|           30|         35|   Y|                N|
|R1QVLVW5QS39ZF|          4|           21|         25|   Y|                N|
| R4SMY6Y2J9897|          5|           24|         28|   Y|                N|
|R1JKVISEGW1GXV|          5|           37|         41|   Y|                N|
|R1ZQ5RKLXUO558|          4|           39|         45|   Y|     

In [8]:
# Step 4. Repeat Step 3 but with "vine" == 'Y'
unpaid_helpful_ttl_g20_vine_df = helpful_ttl_g20_vine_df.filter(helpful_ttl_g20_vine_df["vine"] == "N")
unpaid_helpful_ttl_g20_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2LX6LJ7BA92OO|          1|           21|         27|   N|                Y|
|R2612UZ2GU2K29|          5|           25|         32|   N|                Y|
|R28X8YI8B00C6S|          4|           52|         63|   N|                Y|
|R3O3RDGDHHINF2|          5|           41|         53|   N|                Y|
|R1FSDCDV0XV3X6|          4|           27|         33|   N|                Y|
| RRZXAHEVTT2HC|          5|           69|         92|   N|                Y|
|R3AUV7LA2MZTEO|          5|           16|         21|   N|                N|
| R3NOR8QP5W657|          1|           15|         29|   N|                N|
|R1XM8IQUDFKBQZ|          2|           19|         27|   N|                N|
| RZR9JFL71VIHX|          5|           53|         62|   N|     

In [35]:
# Step 5.1. Count the total of reviews for paid and unpaid.
paid_helpful_ttl_g20_vine = paid_helpful_ttl_g20_vine_df.count()
unpaid_helpful_ttl_g20_vine = unpaid_helpful_ttl_g20_vine_df.count()
ttl_paid_unpaid_helpful_ttl_g20_vine = paid_helpful_ttl_g20_vine_df.count()+unpaid_helpful_ttl_g20_vine_df.count()
print(f'count of paid reviews is {paid_helpful_ttl_g20_vine}, ' 
      f'count of unpaid is {unpaid_helpful_ttl_g20_vine}, ' 
      f'ttl is {ttl_paid_unpaid_helpful_ttl_g20_vine}')

count of paid reviews is 18, count of unpaid is 7652, ttl is 7670


In [10]:
# Step 5.2.1. Count the 5-star reviews for paid.
star5_paid_helpful_ttl_g20_vine = paid_helpful_ttl_g20_vine_df.filter(paid_helpful_ttl_g20_vine_df["star_rating"] == 5).count()
star5_paid_helpful_ttl_g20_vine

9

In [11]:
# Step 5.2.2. Count the 5-star reviews for unpaid.
star5_unpaid_helpful_ttl_g20_vine = unpaid_helpful_ttl_g20_vine_df.filter(unpaid_helpful_ttl_g20_vine_df["star_rating"] == 5).count()
star5_unpaid_helpful_ttl_g20_vine

4376

In [12]:
# Step 5.2.3. Count the 5-star reviews.
star5_paid_unpaid_helpful_ttl_g20_vine = star5_paid_helpful_ttl_g20_vine + star5_unpaid_helpful_ttl_g20_vine
star5_paid_unpaid_helpful_ttl_g20_vine

4385

In [13]:
# Step 5.3. % of 5-star reviews are calculated for "all 5-star paid" and "5-star unpaid" reviews
per_star5_paid_helpful_ttl_g20_vine_2 = (star5_paid_helpful_ttl_g20_vine / star5_paid_unpaid_helpful_ttl_g20_vine) * 100
per_star5_unpaid_helpful_ttl_g20_vine_2 = (star5_unpaid_helpful_ttl_g20_vine / star5_paid_unpaid_helpful_ttl_g20_vine) * 100
print(f'% of "5-star paid" review of "all 5-star" reviews is {per_star5_paid_helpful_ttl_g20_vine_2:.1f}%, \
and % of "5-star unpaid" review of "all 5-star" reviews is {per_star5_unpaid_helpful_ttl_g20_vine_2: .1f}%')

% of "5-star paid" review of "all 5-star" reviews is 0.2%, and % of "5-star unpaid" review of "all 5-star" reviews is  99.8%


In [14]:
# Step 5.4. % of 5-star reviews are calculated for all Vine or non-Vine reviews
per_star5_paid_helpful_ttl_g20_vine = (star5_paid_helpful_ttl_g20_vine / paid_helpful_ttl_g20_vine_df.count()) * 100
per_star5_unpaid_helpful_ttl_g20_vine = (star5_unpaid_helpful_ttl_g20_vine / unpaid_helpful_ttl_g20_vine_df.count()) * 100
print(f'% of "5-star paid" of "all Vine" is {per_star5_paid_helpful_ttl_g20_vine:.1f}%, and % of "5-star unpaid" of non-Vine is {per_star5_unpaid_helpful_ttl_g20_vine: .1f}')

% of "5-star paid" of "all Vine" is 50.0%, and % of "5-star unpaid" of non-Vine is  57.2


In [15]:
# Step 5.5. % 5-star reviews are calculated for all Vine and non-Vine reviews
per_star5_paid_helpful_ttl_g20_vine = (star5_paid_helpful_ttl_g20_vine / ttl_paid_unpaid_helpful_ttl_g20_vine) * 100
per_star5_unpaid_helpful_ttl_g20_vine = (star5_unpaid_helpful_ttl_g20_vine / ttl_paid_unpaid_helpful_ttl_g20_vine) * 100
print(f'% of "5-star paid" of all reviews is {per_star5_paid_helpful_ttl_g20_vine:.1f}%, and % of "5-star unpaid" of all reviews is {per_star5_unpaid_helpful_ttl_g20_vine: .1f}')

% of "5-star paid" of all reviews is 0.1%, and % of "5-star unpaid" of all reviews is  57.1
